<a href="https://colab.research.google.com/github/Reethureddy8/dL-assign2-2/blob/main/Q2_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets evaluate openpyxl --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 8.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82

In [3]:
import pandas as pd

file_path = '/content/Billie_Eilish_Lyrics.xlsx'
df = pd.read_excel(file_path, sheet_name='Lyrics')


In [4]:
import pandas as pd
df = pd.read_excel('Billie_Eilish_Lyrics.xlsx', sheet_name='Lyrics')
df_clean = df[['Lyrics']].dropna().drop_duplicates()
text_data = "\n\n".join(df_clean['Lyrics'].tolist())
with open("lyrics_data.txt", "w", encoding="utf-8") as f:
    f.write(text_data)

In [10]:
from datasets import Dataset
from transformers import GPT2Tokenizer
dataset = Dataset.from_dict({"text": [text_data]})
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
def tokenize_function(examples):
    return tokenizer(examples["text"], return_special_tokens_mask=True)
tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["text"])


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (4721 > 1024). Running this sequence through the model will result in indexing errors


In [11]:
block_size = 128

def group_texts(examples):
    concatenated = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated["input_ids"])
    total_length = (total_length // block_size) * block_size
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated.items()
    }
    return result

lm_dataset = tokenized_dataset.map(group_texts, batched=True)

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [12]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [13]:
import torch
from transformers import GPT2LMHeadModel, DataCollatorForLanguageModeling, Trainer, TrainingArguments
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
training_args = TrainingArguments(
    output_dir="./gpt2-lyrics",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=500,
    save_total_limit=2,
    logging_steps=100,
    prediction_loss_only=True,
    fp16=torch.cuda.is_available(),
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)
trainer.train()

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-13-1573b76ad867>:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss


TrainOutput(global_step=54, training_loss=2.7624523021556713, metrics={'train_runtime': 302.0787, 'train_samples_per_second': 0.358, 'train_steps_per_second': 0.179, 'total_flos': 7054884864000.0, 'train_loss': 2.7624523021556713, 'epoch': 3.0})

In [14]:
trainer.save_model("./gpt2-lyrics")
tokenizer.save_pretrained("./gpt2-lyrics")
print("Model saved to ./gpt2-lyrics")

Model saved to ./gpt2-lyrics


In [15]:
from transformers import pipeline

generator = pipeline("text-generation", model="./gpt2-lyrics", tokenizer=tokenizer)
prompt = "I want it that way"
output = generator(prompt, max_length=100, num_return_sequences=1)[0]['generated_text']
print("Generated Lyrics:\n", output)

Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Generated Lyrics:
 I want it that way, man.

'Bout the bad one,

I just wanna give it a shot

'Cause I never wanna give it a shot

'Cause it's kind of weird I'm not into it



I wanna tell you, right now I don't like how you're judging me"

I wanna tell you, man, maybe I'll never get over that

I kinda wish I could just stay in prison


